In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import scienceplots
import pandas as pd
from itertools import combinations
import pprint
from shapely.geometry import Polygon
plt.style.use(['science', 'grid', 'ieee'])

from utils import (
    load_dataset,
    get_images,
    transform_points,
    get_densities,
    predict,
    calculate_errors,
    get_result_dict,
    SCALING_FACTOR,
    plot_setup, 
    plot_setup_noised,
    plot_predictions,
    add_outlier
)

In [ ]:
def filter_points(points, keys):
    filtered = {key: points[key] for key in keys}
    return filtered

def conduct_experiment(pv_img, tv_img, plot=True, print_errors=True, used_ref_points=None, return_homography=False):
    # Load Data
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    img_pv, img_tv = get_images(pv_img, tv_img)
    
    if used_ref_points:
        reference_pts_pv = filter_points(reference_pts_pv, used_ref_points)
        reference_pts_tv = filter_points(reference_pts_tv, used_ref_points)
        
    # Transform Points to Homogeneous Numpy arrays
    reference_pts_pv_arr, reference_pts_tv_arr, validation_pts_pv_arr, validation_pts_tv_arr = transform_points(
        reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv)
    # Calculate Homography
    h, _ = cv2.findHomography(
        reference_pts_pv_arr,
        reference_pts_tv_arr,
        # method = cv2.RANSAC,
        method = 0,
    )
    h_inv = np.linalg.inv(h)

    # Get Pixel Densities
    reference_densities = get_densities(reference_pts_tv_arr, h_inv)
    validation_densities = get_densities(validation_pts_tv_arr, h_inv)
    
    # Predict Points
    predicted_reference_pts_tv = predict(reference_pts_pv_arr, h)
    predicted_validation_pts_tv = predict(validation_pts_pv_arr, h)
    
    # Errors
    reference_errors = calculate_errors(predicted_reference_pts_tv, reference_pts_tv_arr)
    validation_errors = calculate_errors(predicted_validation_pts_tv, validation_pts_tv_arr)

    # Combine Results in Dictionary
    reference_result_dict = get_result_dict(reference_pts_pv, reference_pts_tv_arr, predicted_reference_pts_tv, reference_errors, reference_densities, reference_pts_pv_arr)
    validation_result_dict = get_result_dict(validation_pts_pv, validation_pts_tv_arr, predicted_validation_pts_tv, validation_errors, validation_densities, reference_pts_pv_arr)
    
    # Print Errors
    if print_errors:
        print(f'Reference  Error: {np.mean(reference_errors) :.2f} PX!')
        print(f'Reference  Error: {np.mean(reference_errors) / SCALING_FACTOR * 100 :.2f} CM!')
        print('---')
        print(f'Validation Error: {np.mean(validation_errors):.2f} PX!')
        print(f'Validation Error: {np.mean(validation_errors) / SCALING_FACTOR * 100 :.2f} CM!')
    
    # Plots
    if plot:
        plot_setup(img_tv, img_pv, reference_result_dict, validation_result_dict)
        plot_predictions(img_tv, img_pv, reference_result_dict, validation_result_dict)
    if return_homography:
        return reference_result_dict, validation_result_dict, h
    return reference_result_dict, validation_result_dict

In [ ]:
def get_dataframe(pv_imgs, tv_img):
    dataframe = pd.DataFrame()
    rows = []
    for pv_img in pv_imgs:
        print(pv_img)
        reference_result_dict, validation_result_dict = conduct_experiment(pv_img, tv_img, plot=False, print_errors=True)
        for pt, pt_dict in reference_result_dict.items():
            pt_dict['name'] = pt
            pt_dict['img'] = pv_img
            pt_dict['point_type'] = 'reference'
            rows.append(pt_dict) 
        for pt, pt_dict in validation_result_dict.items():
            pt_dict['name'] = pt
            pt_dict['img'] = pv_img
            pt_dict['point_type'] = 'validation' 
            rows.append(pt_dict) 
    dataframe = pd.DataFrame(rows)  
    return dataframe
dataframe = get_dataframe(['IMG_01', 'IMG_02', 'IMG_03', 'IMG_04', 'IMG_05', 
                           'IMG_06', 'IMG_07', 'IMG_08', 'IMG_09', 'IMG_10', 
                           'IMG_11', 'IMG_12'], 'IMG_00')

In [ ]:
validation_errors_per_img = dataframe[dataframe['point_type']=='validation'] 
validation_errors_per_img.groupby(['img']).agg(
    mean_error_px = ('error', lambda x: np.mean(x).round(2)),
    mean_error_cm = ('error', lambda x: (np.mean(x)/SCALING_FACTOR*100).round(2)),
    std_cm = ('error', lambda x: (np.std(x)/SCALING_FACTOR*100).round(2)),
    count = ('error', 'count')
)

In [ ]:
np.mean([6.95, 6.93, 5.74, 9.74, 7.67, 65.13, 11.90, 34.03, 8.94, 10.19, 7.59, 8.53]).round(2)